На основании датасета для каждого покупателя вычислите агрегаты, отражающие индивидуальные предпочтения и траты.
Требуемые метрики:

- Предпочитаемый метод оплаты (например, по моде или максимальному обороту).
- Общие траты за весь период наблюдения по покупателю.
- Сумма, потраченная на дополнительные услуги и аксессуары по каждому покупателю.

Ожидается корректная работа кода без ошибок и корректные вычисления по всем клиентам в датасете.

In [2]:
import pandas as pd 
import plotly.express as px
from collections import Counter

# Задание 1

- Идентификатор клиента: уникальный идентификатор каждого покупателя. Customer ID: Unique identifier for each customer.
- Возраст: возраст покупателя (числовое значение). Age: Age of the customer (numeric)
- Пол: пол покупателя (мужской или женский). Gender: Gender of the customer (Male or Female)
- Участник программы лояльности: да/нет; статус меняется со временем, учитывайте, кто отменил и кто подключился. Loyalty Member: (Yes/No) (Values change by time, so pay attention to who cancelled and who signed up)
- Тип продукта: категория электронного товара (например, смартфон, ноутбук, планшет). Product Type: Type of electronic product sold (e.g., Smartphone, Laptop, Tablet)
- SKU: уникальный код каждого товара. SKU: a unique code for each product.
- Рейтинг: оценка товара покупателем от 1 до 5 звёзд; пропусков быть не должно. Rating: Customer rating of the product (1-5 stars) (Should have no Null Ratings)
- Статус заказа: состояние заказа (выполнен, отменён). Order Status: Status of the order (Completed, Cancelled)
- Метод оплаты: способ оплаты (например, наличные, кредитная карта, PayPal). Payment Method: Method used for payment (e.g., Cash, Credit Card, Paypal)
- Итоговая сумма: общая стоимость транзакции (число). Total Price: Total price of the transaction (numeric)
- Цена за единицу: стоимость одной единицы товара (число). Unit Price: Price per unit of the product (numeric)
- Количество: число купленных единиц (число). Quantity: Number of units purchased (numeric)
- Дата покупки: дата совершения покупки (формат: YYYY-MM-DD). Purchase Date: Date of the purchase (format: YYYY-MM-DD)
- Тип доставки: выбранный тип доставки (например, стандарт, ускоренная, экспресс). Shipping Type: Type of shipping chosen (e.g., Standard, Overnight, Express)
- Приобретённые дополнения: список дополнительных позиций (например, аксессуары, расширенная гарантия). Add-ons Purchased: List of any additional items purchased (e.g., Accessories, Extended Warranty)
- Сумма дополнений: общая стоимость приобретённых дополнений (число). Add-on Total: Total price of add-ons purchased (numeric)

In [3]:
df = pd.read_csv('Electronic_sales_Sep2023-Sep2024.csv')
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], format='%Y-%m-%d')

Конечная цена с учетом допов работаем с ней

In [4]:
df["Total_Spent_Row"] = df['Total Price'] + df['Add-on Total']

Оставляем только завершенные заказы

In [5]:
df = df[df['Order Status'] == 'Completed']

## Предпочитаемый метод оплаты (например, по моде или максимальному обороту).

In [6]:
df['Payment Method'] = df['Payment Method'].str.strip().str.lower()

In [7]:
df.groupby('Payment Method', as_index=False)['Total_Spent_Row'].agg({'sum': 'sum', 'count': 'count', 'mean': 'mean'}).sort_values(by='sum', ascending=False)

/tmp/ipykernel_91112/259154179.py:1: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df.groupby('Payment Method', as_index=False)['Total_Spent_Row'].agg({'sum': 'sum', 'count': 'count', 'mean': 'mean'}).sort_values(by='sum', ascending=False)


,Payment Method,sum,count,mean
4,paypal,12915808.46,3863,3343.465819
2,credit card,12828999.89,3899,3290.330826
0,bank transfer,8638680.99,2259,3824.117304
3,debit card,4614956.39,1684,2740.472916
1,cash,4466765.08,1727,2586.430272


In [8]:
df['month'] = df['Purchase Date'].dt.to_period('M').dt.to_timestamp()

In [9]:
df['quartal'] = df['Purchase Date'].dt.to_period('Q').dt.to_timestamp()

In [10]:
monthly_pm = (
    df.groupby(['month', 'Payment Method'], as_index=False)['Total_Spent_Row']
      .sum()
      .rename(columns={'Total_Spent_Row': 'Monthly Sales'})
)

In [11]:
fig = px.line(
    monthly_pm,
    x='month',
    y='Monthly Sales',
    color='Payment Method',
    title='Сумма продаж по месяцам по способам оплаты',
    labels={'month': 'Месяц', 'Monthly Sales': 'Сумма продаж', 'Payment Method': 'Метод оплаты'},
)
fig.update_layout(barmode='stack', legend_title_text='Метод оплаты', xaxis_tickformat='%Y-%m')
fig.show()

## Вывод
Исходя из данных можно сказать, что люди в основном предпочитают использовать пейпал и кредитки с 24 года, эти способы очень близки по количеству и сумме покупок на протяжении всего рассматриваемого периода. 

## Общие траты за весь период наблюдения по покупателю.

In [12]:
df["AgeQuartile"] = pd.qcut(
    df["Age"],
    q=4,
    labels=["Молодые", "Моложе среднего", "Старше среднего", "Старики"]
)

In [13]:
df.groupby(['AgeQuartile', 'Gender'], as_index=False)['Total_Spent_Row'].sum()

/tmp/ipykernel_91112/3867674454.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,AgeQuartile,Gender,Total_Spent_Row
0,Молодые,Female,5563239.56
1,Молодые,Male,5432936.42
2,Моложе среднего,Female,5493146.17
3,Моложе среднего,Male,5540949.21
4,Старше среднего,Female,5433660.78
5,Старше среднего,Male,5744800.39
6,Старики,Female,5190681.90
7,Старики,Male,5065122.06


In [14]:
df.groupby(['Customer ID'], as_index=False)['Total_Spent_Row'].sum().sort_values(by='Total_Spent_Row', ascending=False).head(10)

,Customer ID,Total_Spent_Row
5048,11476,30136.35
7095,15399,29390.27
6177,13635,28481.39
7584,16357,27838.76
5489,12294,27052.70
4960,11332,26965.89
839,2556,26329.06
6124,13534,26261.21
8030,17236,25167.95
6589,14436,25014.73


## Вывод
Исходя из данных можно сказать, что поведение покупателей с возрастом и полом почти не отличается, выбросов почти нет возрастание очень ровное

## Сумма, потраченная на дополнительные услуги и аксессуары по каждому покупателю.

In [15]:
df_addon = df.groupby("Customer ID", as_index=False)["Add-on Total"]\
    .sum()\
    .rename(columns={"Add-on Total": "AddOn_Spent"})\
    .sort_values("AddOn_Spent", ascending=False)\
    


In [16]:
df_addon

,Customer ID,AddOn_Spent
7937,17054,658.32
7221,15648,640.44
5139,11669,613.83
5749,12780,595.63
4926,11270,580.85
...,...,...
4619,9741,0.00
4616,9736,0.00
4609,9713,0.00
4608,9711,0.00


In [17]:
df_addon['AddOn_Spent'].describe()

count    9466.000000
mean       88.273319
std        83.779929
min         0.000000
25%        26.105000
50%        71.490000
75%       127.497500
max       658.320000
Name: AddOn_Spent, dtype: float64

In [18]:
df.groupby(['AgeQuartile', 'Gender'], as_index=False)["Add-on Total"].sum()

/tmp/ipykernel_91112/3163072017.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,AgeQuartile,Gender,Add-on Total
0,Молодые,Female,111940.79
1,Молодые,Male,102129.18
2,Моложе среднего,Female,110208.44
3,Моложе среднего,Male,104497.15
4,Старше среднего,Female,105903.75
5,Старше среднего,Male,108250.62
6,Старики,Female,95759.40
7,Старики,Male,96905.91


In [19]:
fig = px.histogram(
    df_addon[df_addon["AddOn_Spent"] > 0], x="AddOn_Spent",
    nbins=50,
    title="Распределение трат на допы (уровень заказов)",
    labels={"AddOn_Spent": "Сумма допов на заказ"}
)
fig.update_layout(xaxis_tickformat=",")
fig.show()

## Вывод
Исходя из данных можно сказать, допы люди любят брать допы чем моложе тем больше. Само возрастание суммы покупок допов очень ровное квантили растур равномерно.
Сумма в df_addon

# Задание 2

Вычислите показатели выручки (дохода) в разрезах логистики, ассортимента и доп. услуг за разные периоды, используя группировки и ресемплинг/агрегации.
Необходимые разрезы:

- По каждому методу доставки за весь рассматриваемый период.
- По каждому типу продукта с суммированием дохода.
- По дополнительным услугам помесячно для динамики по календарным месяцам.
- По дополнительным услугам поквартально для квартальных трендов.

Постройте визуализации для полученных агрегатов, отобразив их на графиках с корректными подписями осей и легендами.

##  Логистика: по методу доставки

In [20]:
rev_by_shipping = (
df.groupby(["Shipping Type", 'month'], as_index=False)["Total_Spent_Row"]
        .sum()
        .sort_values('month', ascending=False)
)

In [21]:
fig = px.line(
    rev_by_shipping,
    x='month',
    y='Total_Spent_Row',
    color='Shipping Type',
    title='Сумма продаж по месяцам по способам доставки',
    labels={'month': 'Месяц', 'Total_Spent_Row': 'Сумма продаж', 'Shipping Type': 'Метод доставки'},
)
fig.update_layout(barmode='stack', legend_title_text='Метод доставки', xaxis_tickformat='%Y-%m')
fig.show()

## Вывод
Большинство использует стандартный тип доставки начиная с 24 года, до этого все было довольно ровно. Тот же день и экспедиция появились с 24 года и сразу стали популярны.

## По каждому типу продукта с суммированием дохода.

In [22]:
rev_by_product = (
    df.groupby("Product Type", as_index=False)["Total_Spent_Row"]
          .sum()
          .sort_values("Total_Spent_Row", ascending=False)
)
fig_prod = px.bar(
    rev_by_product,
    x="Product Type", y="Total_Spent_Row", text="Total_Spent_Row",
    title="Выручка по типам продуктов (весь период)",
    labels={"Product Type": "Тип продукта", "Total_Spent_Row": "Выручка"}
)
fig_prod.update_traces(texttemplate="%{text:.2f}", textposition="outside")
fig_prod.update_layout(yaxis_tickformat=",", xaxis_tickangle=-30, margin=dict(b=90))
fig_prod.show()

In [32]:
rev_by_product = (
    df.groupby("Product Type", as_index=False)["Quantity"]
          .sum()
          .sort_values("Quantity", ascending=False)
)
fig_prod = px.bar(
    rev_by_product,
    x="Product Type", y="Quantity", text="Quantity",
    title="Выручка по типам продуктов (весь период)",
    labels={"Product Type": "Тип продукта", "Quantity": "Количество покупок"}
)
fig_prod.update_traces(texttemplate="%{text:.2f}", textposition="outside")
fig_prod.update_layout(yaxis_tickformat=",", xaxis_tickangle=-30, margin=dict(b=90))
fig_prod.show()

## Вывод
Самым популярным продуктом по выручке являеются смартфоны, по количеству они тоже всех опережают. А вот, планшетов продаются больше, но по сумме их меньше чем часов и ноутбуков. Что довольно странно. 

## По дополнительным услугам помесячно для динамики по календарным месяцам и кварталам.

In [58]:
rev_by_addon = (
    df.groupby([ "month"], as_index=False)["Add-on Total"]
          .sum()
          .sort_values("month", ascending=False)
)
fig = px.line(
    rev_by_addon,
    x='month',
    y='Add-on Total',
    #color='Add-ons Purchased',
    title='Сумма продаж по месяцам по дополнительным услугам',
    labels={'month': 'Месяц', 'Add-on Total': 'Сумма продаж'},
)
fig.update_layout(barmode='stack', legend_title_text='Дополнительные услуги', xaxis_tickformat='%Y-%m')
fig.show()

In [64]:
# Разбить строку на список категорий
def parse_addons(s):
    if pd.isna(s):
        return []
    # Нормализуем: сплит по запятой, strip пробелов
    items = [x.strip() for x in str(s).split(",") if x.strip()]
    return items

df["AddOn_List"] = df["Add-ons Purchased"].apply(parse_addons)

In [66]:

# Explode: одна строка на каждую категорию допа
df_exploded = df.explode("AddOn_List").reset_index(drop=True)
df_exploded = df_exploded[df_exploded["AddOn_List"].notna() & df_exploded["AddOn_List"].ne("")]

In [67]:
addon_counts = df_exploded["AddOn_List"].value_counts().reset_index()
addon_counts.columns = ["AddOn_Category", "Count"]

In [69]:

fig_freq = px.bar(
    addon_counts,
    x="AddOn_Category", y="Count", text="Count",
    title="Частота категорий допов за весь период",
    labels={"AddOn_Category": "Категория допа", "Count": "Число покупок"}
)
fig_freq.update_traces(texttemplate="%{text}", textposition="outside")
fig_freq.update_layout(xaxis_tickangle=-30, margin=dict(b=90))
fig_freq.show()

In [72]:
rev_by_addon_q = (
    df.groupby([ "quartal"], as_index=False)["Add-on Total"]
          .sum()
          .sort_values("quartal", ascending=False)
)
fig = px.line(
    rev_by_addon_q,
    x='quartal',
    y='Add-on Total',
    #color='Add-ons Purchased',
    title='Сумма продаж по кварталам по дополнительным услугам',
    labels={'quartal': 'Квартал', 'Add-on Total': 'Сумма продаж'},
)
fig.update_layout(barmode='stack', legend_title_text='Дополнительные услуги', xaxis_tickformat='%Y-%m')
fig.show()

In [73]:
rev_by_addon_q

,quartal,Add-on Total
4,2024-07-01,242361.42
3,2024-04-01,256316.84
2,2024-01-01,258222.60
1,2023-10-01,73356.77
0,2023-07-01,5337.61


## Вывод
Резкий взлет продаж с начала 24 года, а дальше стаблильное небольшое падение. Так же равномерно распределены категории доп заказов, нет возможности посчитать точные суммы по ним, но скорее всего они тоже равномерны. 